In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt # data visualization
import seaborn as sns # data visualization

import tensorflow as tf
import numpy as np
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Conv2D, MaxPooling2D, Dropout, BatchNormalization, Flatten, GlobalMaxPooling2D, LeakyReLU
import cv2
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.regularizers import l2
import pandas as pd
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.callbacks import ReduceLROnPlateau

import warnings
warnings.filterwarnings('ignore')

import os
for dirname, _, filenames in os.walk('../kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
df_train = pd.read_csv('../input/Kannada-MNIST/train.csv')
df_test = pd.read_csv('../input/Kannada-MNIST/Dig-MNIST.csv')
y_train = df_train.iloc[:,0].values
y_test = df_test.iloc[:,0].values
df_train = df_train.drop(['label'], axis=1)
df_test = df_test.drop(['label'], axis=1)
N_train = len(y_train)
N_test = len(y_test)


In [ ]:
x_train = np.uint8(df_train.values.reshape(-1,28,28))
x_test = np.uint8(df_test.values.reshape(-1,28,28))

In [ ]:
def preprocessing(img):
  kernel = np.array([[0,-1.5,0],[-1.5,8.5,-1.5],[0,-1.5,0]])
  img = cv2.filter2D(img,-1,kernel)
  img_thresh = img/255.0 *2 -1
  return img_thresh
x_train = np.array(list(map(preprocessing,x_train)))
x_test = np.array(list(map(preprocessing, x_test)))
x_train= x_train.reshape(-1,28,28,1)
x_test= x_test.reshape(-1,28,28,1)
y_train = tf.keras.utils.to_categorical(y_train,10)
y_test = tf.keras.utils.to_categorical(y_test,10)

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_val, Y_train, Y_val = train_test_split(x_train, y_train, random_state=0, test_size=0.15)
X_train.shape, X_val.shape, Y_train.shape, Y_val.shape

In [ ]:
from tensorflow.keras.utils import plot_model
from tensorflow.keras.layers import Add

i = Input(shape=(X_train[0].shape))

x1 = Conv2D(32, (3,3), padding="same")(i)
x1 = LeakyReLU(0.2)(x1)
x1 = BatchNormalization()(x1)
x1 = Conv2D(32, (3,3), padding="same")(x1)
x1 = LeakyReLU(0.2)(x1)
x1 = BatchNormalization()(x1)
x1 = Conv2D(64, (3,3), padding="same")(x1)
x1 = LeakyReLU(0.2)(x1)
b1 = BatchNormalization()(x1)

x2 = Conv2D(64, (3,3), padding="same")(b1)
x2 = LeakyReLU(0.2)(x2)
x2 = BatchNormalization()(x2)
x2 = Conv2D(64, (5,5), padding="same")(x2)
x2 = LeakyReLU(0.2)(x2)
x2 = BatchNormalization()(x2)

x3 = Add()([b1,x2])
x3 = MaxPooling2D()(x3)
x3 = Dropout(0.2)(x3)

x3 = Conv2D(128, (3,3), padding="same")(x3)
x3 = LeakyReLU(0.1)(x3)
x3 = BatchNormalization()(x3)
x3 = Conv2D(128, (3,3), padding="same")(x3)
x3 = LeakyReLU(0.1)(x3)
x3 = Conv2D(256, (3,3), padding="same")(x3)
x3 = LeakyReLU(0.1)(x3)
b2 = BatchNormalization()(x3)

x3 = Conv2D(256, (5,5), padding="same")(b2)
x3 = LeakyReLU(0.1)(x3)
x3 = BatchNormalization()(x3)

x = Add()([b2,x3])

x = MaxPooling2D()(x)
x = Conv2D(256, (3,3), padding="same")(x)
x = LeakyReLU(0.1)(x)
x = GlobalMaxPooling2D()(x)

x = Flatten()(x)
x = Dense(128, activation='relu', kernel_regularizer=l2(0.01))(x)
x = Dropout(0.2)(x)
x = Dense(128, activation='relu', kernel_regularizer=l2(0.01))(x)
x = Dropout(0.2)(x)

x = Dense(10, kernel_regularizer=l2(0.01) )(x)

model = Model(i,x)
#model.summary()
plot_model(model, "model.png", show_shapes=True)

In [ ]:
model.compile(optimizer=Adam(lr=0.001, beta_1=0.9), 
              loss='squared_hinge',
              metrics=['accuracy'])
dataGen = ImageDataGenerator(rotation_range=15,width_shift_range=0.2, height_shift_range=0.2,zoom_range=0.15)
dataGen.fit(X_train)
batch_size=32
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.3,
                              patience=3, min_lr=1e-10)    
r = model.fit_generator(dataGen.flow(X_train, Y_train, batch_size=32),
                        steps_per_epoch=X_train.shape[0]//batch_size, epochs=50, validation_data=(X_val,Y_val), verbose=2,callbacks=[reduce_lr])

In [ ]:
plt.plot(r.history['loss'], label = 'loss')
plt.plot(r.history['val_loss'], label = 'val_loss')
plt.legend()
plt.show()

In [ ]:
model.evaluate(x_test, y_test)

In [ ]:
df_final = pd.read_csv('../input/Kannada-MNIST/test.csv')
df_final = df_final.drop(['id'], axis=1)
X_final = np.uint8(df_final.values.reshape(-1,28,28))
X_final = np.array(list(map(preprocessing,X_final)))
X_final= X_final.reshape(-1,28,28,1)
pred = model.predict(X_final).argmax(axis=1)

In [ ]:
sub = pd.read_csv('../input/Kannada-MNIST/sample_submission.csv')
sub['label'] = pred
sub['label']= sub['label'].astype(int)
sub.to_csv('submission.csv', index=False)